# Installing requirements

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

conn = sqlite3.connect('exp_baseline.db') 

## Baseline results

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Connect to the database and load anomalies for both models
conn = sqlite3.connect('exp_baseline.db')

# Load NAIA anomalies
naia_df = pd.read_sql_query("""
    SELECT * FROM anomalies
    WHERE source_model = 'NAIA'
""", conn, parse_dates=['timestamp'])

# Load LSTM_AE anomalies
lstm_df = pd.read_sql_query("""
    SELECT * FROM anomalies
    WHERE source_model = 'LSTM-AE'
""", conn, parse_dates=['timestamp'])

conn.close()

# Extract hour and date for both DataFrames
for df in (naia_df, lstm_df):
    df['hour'] = df['timestamp'].dt.hour
    df['date'] = df['timestamp'].dt.date

# Determine the last dates for each model (could be NaT if empty)
last_date_naia = naia_df['date'].max()
last_date_lstm = lstm_df['date'].max()

# Build a list of all non-null dates, then take the max
valid_dates = [d for d in (last_date_naia, last_date_lstm) if pd.notnull(d)]
if not valid_dates:
    raise RuntimeError("No anomalies found for either NAIA or LSTM_AE.")
last_date = max(valid_dates)

# Filter to anomalies from that last date for each model
naia_last = naia_df[naia_df['date'] == last_date]
lstm_last = lstm_df[lstm_df['date'] == last_date]

# Prepare a sorted list of all rooms seen by either model on the last date
rooms_naia = set(naia_last['room_id'].unique())
rooms_lstm = set(lstm_last['room_id'].unique())
all_rooms = sorted(rooms_naia.union(rooms_lstm))

# Define all hours (0–23)
all_hours = list(range(24))

# Initialize grids: rows = hours, columns = room IDs
grid_naia = pd.DataFrame(0, index=all_hours, columns=all_rooms)
grid_lstm = pd.DataFrame(0, index=all_hours, columns=all_rooms)

# Mark 1 where an anomaly occurred for NAIA
for _, row in naia_last.iterrows():
    hr = row['hour']
    rm = row['room_id']
    grid_naia.at[hr, rm] = 1

# Mark 1 where an anomaly occurred for LSTM_AE
for _, row in lstm_last.iterrows():
    hr = row['hour']
    rm = row['room_id']
    grid_lstm.at[hr, rm] = 1

# Compute the “flagged by both” grid: 1 only where both models flagged an anomaly
grid_both = (grid_naia & grid_lstm).astype(int)

grid_either = (grid_naia | grid_lstm).astype(int)

print("Grid NAIA")
print(grid_naia)
print("Grid LSTM_AE")
print(grid_lstm)
print("Grid Both Models")
print(grid_both)
print("Grid Either Model")
print(grid_either)

# -----------------------
# Plot: NAIA anomalies
plt.figure(figsize=(12, 4))
plt.imshow(grid_naia.values, aspect='auto', cmap='Reds', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours per room detected by NAIA on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: LSTM_AE anomalies
plt.figure(figsize=(12, 4))
plt.imshow(grid_lstm.values, aspect='auto', cmap='Blues', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours per room detected by LSTM-AE on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Anomalous hours flagged by BOTH models
plt.figure(figsize=(12, 4))
plt.imshow(grid_both.values, aspect='auto', cmap='Purples', origin='lower')
plt.colorbar(label='Anomaly by Both (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours detected by NAIA AND LSTM-AE on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# -----------------------
# Plot: Anomalies flagged by either model
plt.figure(figsize=(12, 4))
plt.imshow(grid_either.values, aspect='auto', cmap='Greys', origin='lower')
plt.colorbar(label='Anomaly (1=Yes)')
plt.xticks(ticks=np.arange(len(all_rooms)), labels=all_rooms, rotation=90)
plt.yticks(ticks=np.arange(24), labels=all_hours)
plt.title(f'Anomalous hours detected by NAIA OR LSTM-AE on{last_date}')
plt.xlabel('Room ID')
plt.ylabel('Hour of Day (0–23)')
plt.tight_layout()
plt.show()

# results statistics
def print_statistics(grid, model_name):
    total_anomalies = grid.values.sum()
    total_rooms = grid.shape[1]
    total_hours = grid.shape[0]
    total_either = grid_either.values.sum()

    
    print(f"\nStatistics for {model_name}:")
    print(f"Total Anomalies Detected: {total_anomalies}")
    print(f"Total Rooms Monitored: {total_rooms}")
    print(f"Total Hours Monitored: {total_hours}")
    print(f"Average Anomalies per Room: {total_anomalies / total_rooms:.2f}")
    print(f"Average Anomalies per Hour: {total_anomalies / total_hours:.2f}")
    print(f"Total Anomalies Detected by Either Model: {total_either}")
    print(f"Percentage of Anomalies detected by {model_name}: "f"{(total_anomalies / total_either) * 100:.2f}%")
# Print statistics for each grid

print_statistics(grid_naia, "NAIA Model")
print_statistics(grid_lstm, "LSTM-AE Model")
print_statistics(grid_both, "Both Models Combined")
print_statistics(grid_either, "Either Model Detected")